In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("moonshotai/Kimi-K2-Instruct",trust_remote_code = True)

In [2]:
tokenizer.encode("Hello workd")

[19180, 1133, 67]

In [3]:
tokenizer.decode([19180, 1133, 67])

'Hello workd'

In [4]:
tokenizer.vocab_size

163842

In [7]:
import torch 
from torch import nn

In [ ]:
class Embeddings(nn.Module):
    